# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import image_lib as imlib
import cv2

# Decision tree classifier

In [3]:
def DTclassification(df, getResult, testSize, pretrainedModel, crossval):

    # Assigning data
    X = df.drop(['label'], axis=1)   
    y = df['label'] 
    
    # Normalisation
    scaler = StandardScaler()
    X_norm = scaler.fit_transform(X)

    # Splitting the data in to training and test data
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size = testSize, random_state=10)
    print('[UPDATE] Data has been loaded successfully!')
    
    if pretrainedModel == 'yes':
        print('[UPDATE] Using pretrained model!')
        DTclassifier = load('PIPE4_DT.joblib')
        print('[UPDATE] Pretrained model loaded!')
    
    else:
        # Identification of hyperparameters
        #param_grid = {
        #    'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        #    'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        #    'min_samples_leaf': [1, 2, 3, 4, 5, 6],
        #    'min_samples_split': [ 8, 12, 16, 18, 20],
        #    }
        
        # The hyperparameters used
        param_grid = {
            'max_depth': [2],
            'max_features': [4],
            'min_samples_leaf': [2],
            'min_samples_split': [ 8],
            }
        
        # Calling the Decision Tree Classifier function
        DecisionTree = tree.DecisionTreeClassifier()
        print('[UPDATE] GridSearch initiated....')
        DTclassifier = GridSearchCV(DecisionTree, param_grid, cv=StratifiedKFold(3))

        # Train model
        print('[UPDATE] Training model....')
        DTclassifier.fit(X_train,y_train)
        print('[UPDATE] Best hyperparameters found in gridsearch: ', DTclassifier.best_params_)
        
        # Saving model
        dump(DTclassifier, 'PIPE4_DT.joblib') 
        print('[UPDATE] Model saved!!! ')
    # Making Prediction
    
    if crossval == 'yes': # Cross Validated Accuracy
        print('[UPDATE] Cross-validating accuracy...')
        scores = cross_val_score(DTclassifier, X_norm, y, cv=StratifiedKFold(10))
        print('------------- Cross validated accuracy ---------------')
        print(scores)
        print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
        print(' ')
        
    
    if getResult == 'yes': # Performance Metrics
        y_pred=DTclassifier.predict(X_norm)
        print('[UPDATE] Printing Results')
        print('------------------ Confusion Matrix -------------------')
        print(confusion_matrix(y, y_pred))
        print(' ')
        print('---------------- Classification report ----------------')
        print(classification_report(y ,y_pred))
        print(' ')
        print('--------------------- AUC score -----------------------')
        print(roc_auc_score(y, y_pred))
    elif getResult == 'no':
        print('[UPDATE] Results not requested')
    else:
        print('[UPDATE] !!!THE REQUEST FOR RESULTS WAS INVALID!!!!')

# Load images, feature extraction, and call decision tree classifier
This code block is what was used to test the the decision tree classifier.

(Import packages and decision tree classifier function before running this)

In [4]:
### pip 1, 2, 4, 5
# Example
augmentation_type = 'No_augmentation_pipeline_4'
pipeline = imlib.pip5
#directory ='darken/' + augmentation_type
path = "/home/sofus/deep/data/test/"

#pos = [cv2.cvtColor(cv2.imread(path + directory + "/Positive/{}_1.jpg".format(i)), cv2.COLOR_BGR2GRAY) for i in range(10502, 14501)]
#neg = [cv2.cvtColor(cv2.imread(path + directory + "/Negative/{}.jpg".format(i)), cv2.COLOR_BGR2GRAY)  for i in range(10502, 14501)]

pos = [cv2.cvtColor(cv2.imread(path + "/Positive/{}_1.jpg".format(i)), cv2.COLOR_BGR2GRAY) for i in range(10502, 14501)]
neg = [cv2.cvtColor(cv2.imread(path + "/Negative/{}.jpg".format(i)), cv2.COLOR_BGR2GRAY)  for i in range(10502, 14501)]

out_p = [pipeline(img) for img in pos]
out_n = [pipeline(img) for img in neg]

feat_pos = [imlib.feature_extraction(img, out_p[i]) for i, img in enumerate(pos)]
feat_neg = [imlib.feature_extraction(img, out_n[i]) for i, img in enumerate(neg)]

SAMPLE_SIZE = len(feat_pos)
print('SAMPLESIZE:', len(feat_pos))

int_mean_p = [feat_pos[i][0] for i in range(len(feat_pos))]
int_mean_n = [feat_neg[i][0] for i in range(len(feat_neg))]
int_mean = np.concatenate((int_mean_p, int_mean_n))

int_stdev_p = [feat_pos[i][1] for i in range(len(feat_pos))]
int_stdev_n = [feat_neg[i][1] for i in range(len(feat_neg))]
int_stdev = np.concatenate((int_stdev_p, int_stdev_n))

ratio_p = [feat_pos[i][2] for i in range(len(feat_pos))]
ratio_n = [feat_neg[i][2] for i in range(len(feat_neg))]
ratio = np.concatenate((ratio_p, ratio_n))

grad_mean_p = [feat_pos[i][3] for i in range(len(feat_pos))]
grad_mean_n = [feat_neg[i][3] for i in range(len(feat_neg))]
grad_mean = np.concatenate((grad_mean_p, grad_mean_n))

grad_mag_p = [feat_pos[i][4] for i in range(len(feat_pos))]
grad_mag_n = [feat_neg[i][4] for i in range(len(feat_neg))]
grad_mag = np.concatenate((grad_mag_p, grad_mag_n))

grad_angle_p = [feat_pos[i][5] for i in range(len(feat_pos))]
grad_angle_n = [feat_neg[i][5] for i in range(len(feat_neg))]
grad_angle = np.concatenate((grad_angle_p, grad_angle_n))

grad_max_p = [feat_pos[i][6] for i in range(len(feat_pos))]
grad_max_n = [feat_neg[i][6] for i in range(len(feat_neg))]
grad_max = np.concatenate((grad_max_p, grad_max_n))

grad_mag_mean_p = [feat_pos[i][7] for i in range(len(feat_pos))]
grad_mag_mean_n = [feat_neg[i][7] for i in range(len(feat_neg))]
grad_mag_mean = np.concatenate((grad_mag_mean_p, grad_mag_mean_n))

grad_stand_p = [feat_pos[i][8] for i in range(len(feat_pos))]
grad_stand_n = [feat_neg[i][8] for i in range(len(feat_neg))]
grad_stand = np.concatenate((grad_stand_p, grad_stand_n))

mean_int_p = [feat_pos[i][9] for i in range(len(feat_pos))]
mean_int_n = [feat_neg[i][9] for i in range(len(feat_neg))]
mean_int = np.concatenate((mean_int_p, mean_int_n))

stdev_mean_p = [feat_pos[i][10] for i in range(len(feat_pos))]
stdev_mean_n = [feat_neg[i][10] for i in range(len(feat_neg))]
stdev_mean = np.concatenate((stdev_mean_p, stdev_mean_n))

# Generating corresponding labels
label_neg = np.array(np.zeros(SAMPLE_SIZE, dtype = int))
label_pos = np.array(np.ones(SAMPLE_SIZE, dtype = int))
labels = np.concatenate((label_pos, label_neg),0)


# Create dataframe

df = pd.DataFrame(np.column_stack([int_mean, int_stdev, ratio, grad_mean, grad_mag, grad_angle, grad_max, 
                                   grad_mag_mean, grad_stand, mean_int, stdev_mean, labels]),  
                  
                  
                  
                  columns=['Local IntensMean', 'Local IntensStDev', 'Ratio', 'GradMean', 
                           'Unique Grad Mag', 'Unique Grad Angle', 'Grad Max', 'Grad Mag Mean', 'Grad Stand',
                           'Global intensMean', 'Global Intens Stand', 'label'])
df.fillna(0, inplace=True)

DTclassification(df, getResult = 'yes', testSize = 0.2051, pretrainedModel = 'yes', crossval = 'yes')
print('----------------- Augmentation type -------------------')
print(augmentation_type)

[UPDATE] Data has been loaded successfully!
[UPDATE] Using pretrained model!
[UPDATE] Pretrained model loaded!
[UPDATE] Cross-validating accuracy...


/data/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/data/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.20.3 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


------------- Cross validated accuracy ---------------
[0.98875    0.99125    0.99375    0.99125    0.985      0.9875
 0.98875    0.9875     0.9925     0.97744361]
Accuracy: 0.9884 (+/- 0.0088)
 
[UPDATE] Printing Results
------------------ Confusion Matrix -------------------
[[3953   46]
 [  34 3965]]
 
---------------- Classification report ----------------
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3999
         1.0       0.99      0.99      0.99      3999

   micro avg       0.99      0.99      0.99      7998
   macro avg       0.99      0.99      0.99      7998
weighted avg       0.99      0.99      0.99      7998

 
--------------------- AUC score -----------------------
0.9899974993748437
----------------- Augmentation type -------------------
No_augmentation_pipeline_4
